In [1]:
import tensorflow as tf
import os
from prettytable import PrettyTable
import numpy as np
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import pad_sequences
from gensim.models import Word2Vec

2024-02-21 19:51:49.814177: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 19:51:49.814363: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 19:51:49.852595: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 19:51:51.209278: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

maxWordsCount = 20000
max_text_len = 100

In [11]:
# # Загрузка существующей модели
# model = Model()
# # os.chdir("/home/dantedanger/LSTMDIPLOME")
# # model_vec = Word2Vec.load("updated_model")
# os.chdir("/home/dantedanger/LSTMDIPLOME")
# model = tf.keras.models.load_model('best_model_f1.h5')

с обучением

In [3]:
os.chdir("/home/dantedanger/LSTMDIPLOME")

with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    texts_true = f.readlines()
    texts_true[0] = texts_true[0].replace('\ufeff', '')
f.close()
with open('train_data_main.txt', 'r', encoding='utf-8') as f:
    texts_main = f.readlines()
    texts_main[0] = texts_main[0].replace('\ufeff', '')
f.close()
with open('train_data_false.txt', 'r', encoding='utf-8') as f:
    texts_false = f.readlines()
    texts_false[0] = texts_false[0].replace('\ufeff', '')
f.close()
print(texts_main[1])

Искусственный интеллект (ИИ) стал важным инструментом в области медицины, привнося с собой ряд новых возможностей и преимуществ. С помощью ИИ, медицинская диагностика и лечение становятся более точными и эффективными.



In [4]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
# patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+«»"
# stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatize(doc):
    patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+«»"
    stopwords_ru = stopwords.words("russian")
    filters='!-"—#$%&amp;,()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»'
    for znak in filters:
        if znak in doc:
            doc = doc.replace(znak, "")  
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru and token != 'с':
            token = token.strip()
            token = token.lower()
            if token == 'ии':
                token = 'искусственый'
                tokens.append(token)
                token = 'интеллект'
                tokens.append(token)
            else: 
                token = morph.normal_forms(token)[0]
                tokens.append(token)
    return tokens

In [5]:
for i in range (len(texts_main)):
    texts_main[i] = lemmatize(texts_main[i])
    texts_true[i] = lemmatize(texts_true[i])
    texts_false[i] = lemmatize(texts_false[i])
print (texts_main[0])

['искусственный', 'интеллект', 'искусственый', 'интеллект', 'стать', 'важный', 'инструмент', 'область', 'медицина', 'привносить', 'себя', 'ряд', 'новый', 'возможность', 'преимущество', 'с', 'помощь', 'искусственый', 'интеллект', 'медицинский', 'диагностика', 'лечение', 'становиться', 'точный', 'эффективный']


In [6]:
all_texts = texts_main + texts_true + texts_false

model_vec = Word2Vec(sentences=all_texts, vector_size=1, window=5, min_count=1, workers=4)

data_main = [[model_vec.wv[token][0] for token in doc] for doc in texts_main]
data_pad_main = pad_sequences(data_main, maxlen=max_text_len, dtype='float32')

data_true = [[model_vec.wv[token][0] for token in doc] for doc in texts_true]
data_pad_true = pad_sequences(data_true, maxlen=max_text_len, dtype='float32')

data_false = [[model_vec.wv[token][0] for token in doc] for doc in texts_false]
data_pad_false = pad_sequences(data_false, maxlen=max_text_len, dtype='float32')
print(len(data_pad_main), len(data_pad_true), len(data_pad_false))
maxWordsCount = model_vec.corpus_total_words

print(data_pad_main)

500 500 500
[[0.         0.         0.         ... 1.4866574  0.25352424 1.88679   ]
 [0.         0.         0.         ... 1.4866574  0.25352424 1.88679   ]
 [0.         0.         0.         ... 1.4866574  0.25352424 1.88679   ]
 ...
 [0.         0.         0.         ... 1.9037777  2.0731452  3.484824  ]
 [0.         0.         0.         ... 1.9037777  2.0731452  3.484824  ]
 [0.         0.         0.         ... 1.9037777  2.0731452  3.484824  ]]


In [8]:
X = []
Y = []

for main_row, main_row2, true_row, false_row in zip(data_pad_main, data_pad_main, data_pad_true, data_pad_false):
    X.append([main_row, true_row])
    Y.append([0.8])

    X.append([main_row2, false_row])
    Y.append([0])

X = np.array(X)
Y = np.array(Y)
print(X[:, 0], X[:, 1])

[[0.         0.         0.         ... 1.4866574  0.25352424 1.88679   ]
 [0.         0.         0.         ... 1.4866574  0.25352424 1.88679   ]
 [0.         0.         0.         ... 1.4866574  0.25352424 1.88679   ]
 ...
 [0.         0.         0.         ... 1.9037777  2.0731452  3.484824  ]
 [0.         0.         0.         ... 1.9037777  2.0731452  3.484824  ]
 [0.         0.         0.         ... 1.9037777  2.0731452  3.484824  ]] [[ 0.          0.          0.         ...  2.843095   -0.6356724
   4.733336  ]
 [ 0.          0.          0.         ...  3.8304741   3.1616075
   0.491813  ]
 [ 0.          0.          0.         ...  1.0697935   2.666802
   2.812623  ]
 ...
 [ 0.          0.          0.         ...  0.81307393  0.5917751
   3.503475  ]
 [ 0.          0.          0.         ...  0.7930694   3.1616075
   2.3951287 ]
 [ 0.          0.          0.         ...  4.733336    0.9595525
   2.812623  ]]


In [9]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.5, **kwargs):
        super(F1Score, self).__init__(**kwargs)
        self.threshold = threshold
        self.tp = self.add_weight('true_positives', initializer='zeros')
        self.fp = self.add_weight('false_positives', initializer='zeros')
        self.fn = self.add_weight('false_negatives', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred >= self.threshold, tf.float32)
        y_true = tf.cast(y_true, tf.float32)

        true_positives = tf.reduce_sum(y_true * y_pred)
        false_positives = tf.reduce_sum(y_pred) - true_positives
        false_negatives = tf.reduce_sum(y_true) - true_positives

        self.tp.assign_add(true_positives)
        self.fp.assign_add(false_positives)
        self.fn.assign_add(false_negatives)

    def result(self):
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        return f1

    def reset_state(self):
        self.tp.assign(0)
        self.fp.assign(0)
        self.fn.assign(0)

In [10]:
from keras.callbacks import ModelCheckpoint
os.chdir("/home/dantedanger/LSTMDIPLOME")
checkpoint_f1 = ModelCheckpoint("best_model_f1.h5", monitor="val_f1_score", save_best_only=True, mode="max")
checkpoint_mse = ModelCheckpoint("best_model_mse.h5", monitor="val_loss", save_best_only=True, mode="min")
checkpoint_f1_main = ModelCheckpoint("best_model_f1_main.h5", monitor="f1_score", save_best_only=True, mode="max")
checkpoint_mse_main = ModelCheckpoint("best_model_mse_main.h5", monitor="loss", save_best_only=True, mode="min")

In [11]:
# from keras.layers import Lambda
from keras.layers import Reshape
# import keras.backend as K

from keras.layers import Dot
from keras.layers import Input, LSTM, Embedding

model = Model()

input_main_text1 = Input(shape=(max_text_len,))
input_main_text2 = Input(shape=(max_text_len,))

# embedding_layer = Embedding(maxWordsCount, 128, input_length=max_text_len)
# embedded_main_text = embedding_layer(input_main_text1)
# embedded_related_text = embedding_layer(input_main_text2)

reshaped_input_main_text1 = Reshape((100, 1))(input_main_text1)
reshaped_input_main_text2 = Reshape((100, 1))(input_main_text2)

# LSTM слой для каждой текстовой последовательности
lstm_layer = LSTM(128, activation='tanh')
lstm_main_text1 = lstm_layer(reshaped_input_main_text1)
lstm_main_text2 = lstm_layer(reshaped_input_main_text2)

cosine_similarity = Dot(axes=(1), normalize=True)([lstm_main_text1, lstm_main_text2])

# Функция для вычисления косинусного расстояния
# def cosine_distance(vects):
#     x, y = vects
#     x = K.l2_normalize(x, axis=-1)
#     y = K.l2_normalize(y, axis=-1)
#     return K.sum(x * y, axis=-1, keepdims=True)

# cosine_similarity = Lambda(cosine_distance, output_shape=(1,))([lstm_main_text1, lstm_main_text2])

# Слой для расчета косинусного расстояния между выходами LSTM слоев
# cosine_similarity = Lambda(cosine_distance, output_shape=(1,))([lstm_main_text1, lstm_main_text2])

# Создание модели
model = Model(inputs=[input_main_text1, input_main_text2], outputs=cosine_similarity)

model.summary()

model.compile(loss='mean_squared_error', optimizer=SGD(learning_rate=0.01, momentum=0.9)
               ,metrics=[F1Score()]
              )

# Обучение модели
history = model.fit([X[:, 0], X[:, 1]], Y, batch_size=1, epochs=150, validation_split=0.02, callbacks=[checkpoint_f1, checkpoint_mse, checkpoint_f1_main, checkpoint_mse_main])

2024-02-21 19:57:31.340795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 19:57:31.454240: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 19:57:31.454338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 19:57:31.459512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 19:57:31.459627: I tensorflow/compile

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 reshape (Reshape)           (None, 100, 1)               0         ['input_1[0][0]']             
                                                                                                  
 reshape_1 (Reshape)         (None, 100, 1)               0         ['input_2[0][0]']             
                                                                                            

2024-02-21 19:57:43.200722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-02-21 19:57:49.287872: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3c306a8550 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-21 19:57:49.287974: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX130, Compute Capability 5.0


  3/980 [..............................] - ETA: 1:22 - loss: 0.0380 - f1_score: 0.8889 

2024-02-21 19:57:49.619893: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


980/980 [==============================] - 56s 42ms/step - loss: 0.3141 - f1_score: 0.5741 - val_loss: 0.3585 - val_f1_score: 0.5714
Epoch 2/150


/home/dantedanger/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


980/980 [==============================] - 37s 38ms/step - loss: 0.2424 - f1_score: 0.5481 - val_loss: 0.2758 - val_f1_score: 0.5565
Epoch 3/150
980/980 [==============================] - 42s 43ms/step - loss: 0.2055 - f1_score: 0.5419 - val_loss: 0.2865 - val_f1_score: 0.5565
Epoch 4/150
980/980 [==============================] - 39s 40ms/step - loss: 0.2389 - f1_score: 0.5647 - val_loss: 0.3092 - val_f1_score: 0.5120
Epoch 5/150
980/980 [==============================] - 41s 41ms/step - loss: 0.1686 - f1_score: 0.5978 - val_loss: 0.3290 - val_f1_score: 0.5714
Epoch 6/150
980/980 [==============================] - 42s 43ms/step - loss: 0.1729 - f1_score: 0.5735 - val_loss: 0.2267 - val_f1_score: 0.5333
Epoch 7/150
980/980 [==============================] - 41s 42ms/step - loss: 0.1475 - f1_score: 0.6208 - val_loss: 0.1747 - val_f1_score: 0.4267
Epoch 8/150
980/980 [==============================] - 42s 43ms/step - loss: 0.1572 - f1_score: 0.6088 - val_loss: 0.2684 - val_f1_score: 0.57

In [17]:
# Загрузка существующей модели
model = Model()
# os.chdir("/home/dantedanger/LSTMDIPLOME")
# model_vec = Word2Vec.load("updated_model")
os.chdir("/home/dantedanger/LSTMDIPLOME")
model = tf.keras.models.load_model('best_model_f1.h5')
# model = tf.keras.models.load_model('best_model_f1_main.h5')
# model = tf.keras.models.load_model('best_model_mse.h5')
# model = tf.keras.models.load_model('best_model_mse_main.h5')

In [18]:
os.chdir("/home/dantedanger/LSTMDIPLOME/Education_list")
files = os.listdir()
col = len(files) + 1
print(files)
a = [[""] * col for i in range(col)]
for n, item1 in enumerate(files):
    with open(item1, 'r', encoding='utf-8') as f:
        main_text = f.readline()
        new_main_text = f.readlines()
        a[n + 1][0] = main_text
    for k, item in enumerate(files):
        if k < n:
            with open(item, 'r', encoding='utf-8') as f1:
                related_text = f1.readline()
                new_related_text = f1.readlines()
                a[0][k + 1] = related_text
            
            new_new_main_text = lemmatize(new_main_text[0])
            new_new_related_text = lemmatize(new_related_text[0])

            print(new_new_main_text, new_new_related_text, sep="\n")

            two_texts = [new_new_main_text + new_new_related_text]

            # Добавление новых текстов к существующей модели
            model_vec.build_vocab(two_texts, update=True)

            # Переобучение модели на расширенном корпусе данных
            model_vec.train(two_texts, total_examples=model_vec.corpus_count, epochs=model_vec.epochs)

            new_main_sequence = [model_vec.wv[token][0] for token in new_new_main_text]
            new_padded_main_sequence = pad_sequences([new_main_sequence], maxlen=max_text_len, dtype='float32')

            new_related_sequence = [model_vec.wv[token][0] for token in new_new_related_text]
            new_padded_related_sequence = pad_sequences([new_related_sequence], maxlen=max_text_len, dtype='float32')

            new_X = [new_padded_main_sequence, new_padded_related_sequence]
            prediction = model.predict(new_X)

            predicted_label = "+" if prediction[0][0] >= 0.4 else "-"
            a[n + 1][k + 1] = predicted_label
            a[k + 1][n + 1] = predicted_label
            print("", main_text, related_text, predicted_label, prediction[0][0])

            f.close()
            f1.close()
        else:
            break
a[0][col - 1] = a[col - 1][0]

['main.txt', 'Б1.Б9_Физика.txt', 'Б1.Б2_Алгебра_и_аналитическая_геометрия.txt', 'Б1.Б5_История.txt', 'Б1.Б8_Русский_язык_и_культура_речи.txt', 'Б1.ДВ6.1_Нейрокомпьютерные_системы.txt', 'Б1.В16_Правоведние.txt', 'false.txt', 'Б1.В1_Дискретная_математика.txt', 'true.txt', '03.09.04_Иностранный_яызк.txt', 'Б1.Б3_Математический_анализ.txt']
['дисциплина', 'физика', 'б1б9', 'входить', 'обязательный', 'часть', 'блок', 'б1', 'дисциплина', 'модуль', 'знание', 'умение', 'навык', 'получить', 'ход', 'изучение', 'дисциплина', 'являться', 'необходимый', 'изучение', 'дисциплина', 'химия', 'экология', 'раздел', '1', 'физический', 'основа', 'механика', 'элемент', 'кинематика', 'физический', 'модель', 'материальный', 'точка', 'система', 'материальный', 'точка', 'абсолютно', 'твёрдый', 'тело', 'пространство', 'время', 'описание', 'движение', 'прямолинейный', 'движение', 'точка', 'скорость', 'ускорение', 'движение', 'точка', 'окружность', 'угловой', 'скорость', 'угловой', 'ускорение', 'скорость', 'ускоре

In [19]:
table = PrettyTable()
for row in a:
    table.add_row(row, divider=True)
table.header = False
print(table)

+-----------------------------------------+------+--------------+-----------------------------------------+---------------+------------------------------------+--------------------------------------+--------------------+-------+-----------------------------+------+---------------------------+-----------------------------+
|                                         | main | Б1.Б9 Физика | Б1.Б2 Алгебра и аналитическая геометрия | Б1.Б5 История | Б1.Б8 Русский язык и культура речи | Б1.В.ДВ6.1 Нейрокомпьютерные системы | Б1.В16 Правоведние | false | Б1.В1 Дискретная математика | true | 03.09.04 Иностранный язык | Б1.Б3 Математический анализ |
|                                         |      |              |                                         |               |                                    |                                      |                    |       |                             |      |                           |                             |
+---------------------------

In [103]:
# Сохранение модели
os.chdir("/home/dantedanger/LSTMDIPLOME")
model_vec.save("updated_model")
model.save('my_model.keras')
model.save_weights('weights')